In [67]:
import pandas as pd
import numpy as np

In [68]:
cols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID']

In [69]:
data_iter = pd.read_csv('C:/Users/Konrad/Downloads/2018_Yellow_Taxi_Trip_Data.csv', usecols = cols, iterator = True, chunksize = 10000)

In [72]:
for df in data_iter:
    df.rename(columns = {'PULocationID': 'pickup_location', 'DOLocationID': 'dropoff_location'}, inplace = True)

    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    df['day_of_week_pickup_sin'] = np.sin(df['tpep_pickup_datetime'].dt.dayofweek * (2 * np.pi / 7))
    df['day_of_week_pickup_cos'] = np.cos(df['tpep_pickup_datetime'].dt.dayofweek * (2 * np.pi / 7))

    df['day_of_week_dropoff_sin'] = np.sin(df['tpep_dropoff_datetime'].dt.dayofweek * (2 * np.pi / 7))
    df['day_of_week_dropoff_cos'] = np.cos(df['tpep_dropoff_datetime'].dt.dayofweek * (2 * np.pi / 7))

    df['pickup_time'] = df['tpep_pickup_datetime'].dt.time
    df['dropoff_time'] = df['tpep_dropoff_datetime'].dt.time

    df.drop(columns = ['tpep_pickup_datetime', 'tpep_dropoff_datetime'], inplace = True)

    df.to_csv('2018_Yellow_Taxi_Trip_Data_Clean.csv', mode = 'a', index = False)

KeyboardInterrupt: 